In [13]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from scipy.spatial.transform import Rotation as R

In [24]:
import tensorflow as tf
from tensorflow.keras import Sequential, Input
from tensorflow.keras.layers import (
    Conv1D, BatchNormalization, Activation,
    MaxPooling1D, Dropout, Flatten, Dense
)
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import KFold
from sklearn.utils import compute_class_weight

In [15]:
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [25]:
np_data = 'stroke_peak_data.npz'
batch_size = 128
num_epochs = 256
patience = 50
#seed = 42
seed = np.random.seed()
rng = np.random.default_rng(seed=seed)
test_split = 0.2
val_split = 0.2

In [26]:
# Load data
data = np.load(np_data)
X = data['X']
y = data['y']
X = X.astype(np.float16)
y = y.astype(np.int32)

print(f"Total samples: {X.shape[0]}, each window shape: {X.shape[1:]}")
print(f"Labels shape: {y.shape}")

Total samples: 658, each window shape: (64, 6)
Labels shape: (658,)


In [27]:
# Split data into train, validation, and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=test_split,
    random_state=seed,
    stratify=y
)

num_classes = len(set(y_train))

print("After split:")
print(f"  X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"  X_test : {X_test.shape},  y_test : {y_test.shape}")

After split:
  X_train: (526, 64, 6), y_train: (526,)
  X_test : (132, 64, 6),  y_test : (132,)


In [28]:
# Shuffle and rotate data
def random_rotate_sample(sample, label):
    def rotate_fn(sample_np):
        random_rotation = R.random(rng=rng)
        rotated_accel = random_rotation.apply(sample_np[:, :3])
        rotated_gyro = random_rotation.apply(sample_np[:, 3:])
        rotated_sample = np.concatenate((rotated_accel, rotated_gyro), axis=1)
        return rotated_sample.astype(np.float16)

    rotated_sample = tf.py_function(func=rotate_fn, inp=[sample], Tout=tf.float16)
    rotated_sample.set_shape(sample.shape)
    return rotated_sample, label

test_ds = (
    tf.data.Dataset.from_tensor_slices((X_test, y_test))
    .batch(batch_size)
    .prefetch(tf.data.AUTOTUNE)
)

In [29]:
time_steps, n_feats = X_train.shape[1], X_train.shape[2]   # 64, 6

def random_rotate_sample(sample, label):
    # sample: tf.Tensor, shape=(time_steps, n_feats), dtype=tf.float16
    # label:  tf.Tensor, shape=(), dtype=tf.int32 (or float32 if one-hot)

    def rotate_fn(sample_np):
        # sample_np: np.ndarray, shape=(time_steps, n_feats), dtype=float16
        # cast to float64 for scipy
        sample_np = sample_np.astype(np.float64)

        # generate one random 3D rotation
        rot = R.random(random_state=rng)

        # apply to accel (first 3 channels) and gyro (last 3 channels)
        accel_rot = rot.apply(sample_np[:, :3])   # (time_steps, 3)
        gyro_rot  = rot.apply(sample_np[:, 3:])   # (time_steps, 3)

        # stitch back together and cast to float32
        out = np.concatenate([accel_rot, gyro_rot], axis=1)
        return out.astype(np.float16)

    rotated = tf.numpy_function(
        func=rotate_fn,
        inp=[sample],
        Tout=tf.float16
    )

    rotated.set_shape([time_steps, n_feats])
    return rotated, label

test_ds = (
    tf.data.Dataset.from_tensor_slices((X_test, y_test))
    .batch(batch_size)
    .prefetch(tf.data.AUTOTUNE)
)

In [30]:
# Conv1D Model
def build_model():
    model = Sequential([
        Input(shape=(64, 6)),

        Conv1D(64, kernel_size=3, padding='same'),
        BatchNormalization(),
        Activation('relu'),
        Dropout(0.1),

        Conv1D(128, kernel_size=3, padding='same'),
        BatchNormalization(),
        Activation('relu'),
        MaxPooling1D(pool_size=2),
        Dropout(0.1),

        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.4),
        Dense(64, activation='relu'),
        Dropout(0.4),
        Dense(num_classes, activation='softmax')
    ])

    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model
#model.summary()

In [32]:
kf = KFold(n_splits=5, shuffle=True, random_state=seed)
val_accuracies = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X_train, y_train)):
    print(f"\n--- Fold {fold+1} ---")
    X_tr, X_va = X_train[train_idx], X_train[val_idx]
    y_tr, y_va = y_train[train_idx], y_train[val_idx]

    # Convert np arrays to Tensors
    train_ds = (
        tf.data.Dataset.from_tensor_slices((X_tr, y_tr))
        .shuffle(len(X_tr), seed=seed)
        .map(random_rotate_sample, num_parallel_calls=tf.data.AUTOTUNE)
        .batch(batch_size)
        .prefetch(tf.data.AUTOTUNE)
    )
    val_ds = (
        tf.data.Dataset.from_tensor_slices((X_va, y_va))
        .batch(batch_size)
        .prefetch(tf.data.AUTOTUNE)
    )

    # Build a fresh model and callbacks for each fold
    model = build_model()
    early = EarlyStopping(monitor='val_loss', patience=patience, restore_best_weights=True)
    ckpt = ModelCheckpoint(
        f"best_fold_{fold}.keras",
        monitor="val_loss",
        save_best_only=True,
        save_weights_only=False,   # save the full model
    )

    # Comput class weights
    class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
    class_weights = {i: class_weights[np.where(np.unique(y_train) == i)[0][0]] if i in np.unique(y_train) else 0 for i in range(num_classes)}


    # Train
    history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=num_epochs,
        batch_size=batch_size,
        callbacks=[early, ckpt],
        class_weight=class_weights,
        verbose=0
    )

    # load back the best weights for this fold and evaluate on val set
    best = tf.keras.models.load_model(f"best_fold_{fold}.keras")
    loss, acc = best.evaluate(val_ds, verbose=0)
    print(f"Fold {fold} best val_acc = {acc:.4f}")
    val_accuracies.append(acc)

# Aggregate results
val_accuracies = np.array(val_accuracies)
print(f"\nMean val accuracy: {val_accuracies.mean():.4f} ± {val_accuracies.std():.4f}")

test_loss, test_acc = model.evaluate(test_ds)
print(f"Test accuracy: {test_acc:.3f}")


--- Fold 1 ---
Fold 0 best val_acc = 0.8019

--- Fold 2 ---
Fold 1 best val_acc = 0.8095

--- Fold 3 ---
Fold 2 best val_acc = 0.7714

--- Fold 4 ---
Fold 3 best val_acc = 0.7429

--- Fold 5 ---
Fold 4 best val_acc = 0.8190

Mean val accuracy: 0.7889 ± 0.0280
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 265ms/step - accuracy: 0.8267 - loss: 0.5825
Test accuracy: 0.818


In [33]:
val_scores = np.array(val_accuracies)
best_fold = np.argmax(val_accuracies) + 1
print(f"Best fold: {best_fold} with val_acc = {val_accuracies[best_fold-1]:.4f}")

# that file is:
best_filepath = f"best_fold_{best_fold}.keras"
best_filepath

Best fold: 5 with val_acc = 0.8190


'best_fold_5.keras'

In [35]:
# Training

train_ds = (
        tf.data.Dataset.from_tensor_slices((X_train, y_train))
        .shuffle(len(X_tr), seed=seed)
        .map(random_rotate_sample, num_parallel_calls=tf.data.AUTOTUNE)
        .batch(batch_size)
        .prefetch(tf.data.AUTOTUNE)
    )

# Build a fresh model and callbacks for each fold
model = build_model()
early = EarlyStopping(monitor='loss', patience=patience, restore_best_weights=True)
ckpt  = ModelCheckpoint(best_filepath, monitor="loss", save_best_only=True)

# Comput class weights
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights = {i: class_weights[np.where(np.unique(y_train) == i)[0][0]] if i in np.unique(y_train) else 0 for i in range(num_classes)}

# Train
history = model.fit(
    train_ds,
    #validation_data=val_ds,
    epochs=num_epochs,
    batch_size=batch_size,
    callbacks=[early, ckpt],
    class_weight=class_weights,
    verbose=1
)

test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(f"Test accuracy: {test_acc:.3f}")

Epoch 1/256
5/5 ━━━━━━━━━━━━━━━━━━━━ 9s 867ms/step - accuracy: 0.0988 - loss: 3.2822
Epoch 2/256
5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 58ms/step - accuracy: 0.0970 - loss: 2.7876
Epoch 3/256
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.1292 - loss: 2.5564
Epoch 4/256
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 0.1387 - loss: 2.5227
Epoch 5/256
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.1391 - loss: 2.4547
Epoch 6/256
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.1592 - loss: 2.3890
Epoch 7/256
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.1444 - loss: 2.4066
Epoch 8/256
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.1922 - loss: 2.4047
Epoch 9/256
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 58ms/step - accuracy: 0.1538 - loss: 2.3940
Epoch 10/256
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.1599 - loss: 2.3284
Epoch 11/256
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.1839 - loss: 2.3459
Epoch 12/256
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step - accuracy: 0.1865 - l

In [ ]:
# Convert to TFLite
def representative_dataset():
    for input_value in tf.data.Dataset.from_tensor_slices(X_train).batch(1).take(100):
        # ensure dtype matches your input dtype (float32 here)
        yield [input_value]

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset

# For full integer quantization (weights + activations)
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type  = tf.uint8   # or tf.int8
converter.inference_output_type = tf.uint8   # match input type

tflite_model = converter.convert()
open('conv1d_model.tflite', 'wb').write(tflite_model)
print("TFLite model written to conv1d_model.tflite")